## Self-Licensing

In [ ]:
import pandas as pd
import os
import re
import numpy as np
import pandas as pd
import pickle
from collections import Counter, defaultdict
import math
import json
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from main_analyze import get_main_data
from statsmodels.stats import weightstats as stests
df_all = get_main_data(assign_net_income_rank_flag=True)

In [ ]:
import seaborn as sns 
sns.set(font_scale=1.4) 
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
g = sns.catplot(data=df_all.query('year>=1990') , kind="bar", x="year", y="self-licensed", ci=None, color='royalblue')
g.set_xticklabels(rotation=90)
g.set(ylabel="Fraction Licensed by Inventor Startups")
for ax in g.axes.flat:
    for ind, label in enumerate(ax.get_xticklabels()):
        if ind % 2 == 0:  
            label.set_visible(True)
        else:
            label.set_visible(False)

In [ ]:
import statsmodels.api as sm
from main_analyze import label_vocab
interested_label_vocab = label_vocab[:6] 
func_name = 'self-licensed'
X = df_all[['year'] + interested_label_vocab] 
Y = df_all[func_name] 
X = sm.add_constant(X) 
model = sm.OLS(Y, X.astype(float)).fit()
predictions = model.predict(X) 
print_model = model.summary()
print(print_model)
print('p-values:', model.pvalues)

In [ ]:
net_income_range_str_list = list()
self_licensed_rate_list = list()
label_query_scheme = [
    [r'<\$10K',         '`Net Income`<1e4', ],
    [r'\$10K-\$100K',   '`Net Income`>=1e4 and `Net Income`<=1e5',],
    [r'\$100K-\$1M',    '`Net Income`>=1e5 and `Net Income`<=1e6',],
    [r'\$1M-\$10M',     '`Net Income`>=1e6 and `Net Income`<=1e7',],
    [r'≥\$10M',        '`Net Income`>=1e7 ',],
]
for label_str, query_str in label_query_scheme:
    df_range = df_all.query( query_str )
    self_licensed_rate = df_range['self-licensed'].mean()
    range_str = label_str
    net_income_range_str_list.append(range_str)
    self_licensed_rate_list.append(self_licensed_rate)
    print(range_str, self_licensed_rate, 'Sample size:', len(df_range))
df_tmp = {
    'net income': net_income_range_str_list, 
    'self-licensing rate': self_licensed_rate_list,
}
import seaborn as sns 
sns.set() 
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
g = sns.barplot(x=net_income_range_str_list, y=self_licensed_rate_list)
g.set(xlabel='Net income (USD)', ylabel="Fraction Licensed by Inventor Startups")

In [ ]:
import statsmodels.api as sm
from main_analyze import label_vocab
interested_label_vocab = label_vocab[:6] 
X = df_all[['year', 'self-licensed'] + interested_label_vocab] 
Y = df_all['income_rank'] 
X = sm.add_constant(X) 
model = sm.OLS(Y, X.astype(float)).fit()
predictions = model.predict(X) 
print_model = model.summary()
print(print_model)
print('p-values:', model.pvalues)